# Prueba

In [ ]:
import pandas as pd
import statsmodels as ss
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import missingno as msngo
pd.options.plotting.backend = "plotly"

## Pregunta 1

Describe con detenimiento el problema de negocio que se desea resolver, y cómo se va a hacer esto. Indica la metodología, tareas a realizar, variable objetivo a predecir,
etc.

Pensando en una metodologia Crisp-DM se piensa en un entendimiento del negocio, en este caso de venta de pasajes de avion, pensando en esto es conocido que variadas cosas son las que influyen, como la hora y el dia del vuelo, tipo de pasaje, el tiempo que toma, si tienen alguna parada, y obviamente el origen y el destino, estos datos son los que contamos en el dataset entregado

In [ ]:
df_business = pd.read_excel('Data/business.xlsx')
df_business.head()

In [ ]:
df_economy = pd.read_excel('Data/economy.xlsx')
df_economy.head()

Este es un problema de prediccion de precios de vuelos, se presentan dos dataframes, ambos contien la misma informacion pero para dos tipos de pasaje de avion, uno economy y el otro para business, dado esto tienen que unirse los dataframes en uno solo creando la columna de tipo de boleto.

## Pregunta 2

Carga los dataset entregados y genera un reporte de calidad de los mismos. Indica qué estrategias se van a utilizar para aquellos puntos encontrados (Indicar nulos,
outliers, valores perdidos, que se hará con esto, etc.)

Para la resolucion del problema se haran algunas transformaciones como:
- Convertir el tiempo de viaje a minutos, 
- Separar el tiempo de salida y de llegada en 3 periodos, mañana tarde y noche
- Convertir la fecha del viaje en dia de la semana
- La columna num_code se eliminará ya que no se tienen

In [ ]:
df_business.drop(columns=['num_code'], inplace=True)
df_economy.drop(columns=['num_code'], inplace=True)

In [ ]:
msngo.matrix(df_business)

In [ ]:
msngo.matrix(df_economy)

No se observan nulos en ninguno de los dataframes

In [ ]:
df_business.info(True)

In [ ]:
df_economy.info(True)

In [ ]:
airline_code = df_economy[['airline', 'ch_code']].drop_duplicates()
airline_code.head(20)

In [ ]:
airline_code = df_business[['airline', 'ch_code']].drop_duplicates()
airline_code.head(20)

Con lo anterior se confirma que CH_CODE es una columna que hace referencia a la aerolinea, por lo que se dropeara airline

In [ ]:
df_business.drop(columns=['airline'], inplace=True)
df_economy.drop(columns=['airline'], inplace=True)

In [ ]:
display(df_business.stop.unique())
display(df_economy.stop.unique())

En la columna Stop vemos que contiene informacion de si tuvo paradas el vuelo o no, en este caso vemos que tiene la informacion muy sucia y que hay registros que contienen la via en que realizaron la parada, y otros no, dada la falta de esta informacion se tomara en cuenta el numero de paradas que tiene el vuelo y se dejara como una columna

In [ ]:
df_business.stop = df_business.stop.apply(lambda x: 0 if x == 'non-stop' else int(x[0]))
df_economy.stop = df_economy.stop.apply(lambda x: 0 if x == 'non-stop' else int(x[0]))

In [ ]:
df_business.date = df_business.date.apply(lambda x: x.date().strftime("%A"))
df_economy.date = df_economy.date.apply(lambda x: x.date().strftime("%A"))

In [ ]:
df_economy['hours'] = df_economy.time_taken.apply(lambda x: x[:x.index('h')])
df_economy['minutes'] = df_economy.time_taken.apply(lambda x: x[x.index('h')+1:x.index('m')])
df_business['hours'] = df_business.time_taken.apply(lambda x: x[:x.index('h')])
df_business['minutes'] = df_business.time_taken.apply(lambda x: x[x.index('h')+1:x.index('m')])

df_business.time_taken = df_business.hours.apply(lambda x: int(float(x)*60) if x.strip() != '' else 0) + \
    df_business.minutes.apply(lambda x: int(x) if x.strip() != '' else 0)

df_economy.time_taken = df_economy.hours.apply(lambda x: int(float(x)*60) if x.strip() != '' else 0) + \
    df_economy.minutes.apply(lambda x: int(x) if x.strip() != '' else 0)
df_economy.drop(columns=['hours', 'minutes'], inplace=True)
df_business.drop(columns=['hours', 'minutes'], inplace=True)

In [ ]:
def categorizar_hora(hora):
    if hora >= pd.to_datetime('00:00:00').time() and hora < pd.to_datetime('06:00:00').time():
        return 'madrugada'
    elif hora >= pd.to_datetime('06:00:00').time() and hora < pd.to_datetime('12:00:00').time():
        return 'manana'
    elif hora >= pd.to_datetime('12:00:00').time() and hora < pd.to_datetime('18:00:00').time():
        return 'tarde'
    else:
        return 'noche'

# Aplicar la función a la columna 'hora'
df_economy['dep_timeday'] = df_economy.dep_time.apply(categorizar_hora)
df_business['dep_timeday'] = df_business.dep_time.apply(categorizar_hora)
df_economy.drop(columns=['dep_time'], inplace=True)
df_business.drop(columns=['dep_time'], inplace=True)

In [ ]:
df_economy['arr_timeday'] = df_economy.arr_time.apply(categorizar_hora)
df_business['arr_timeday'] = df_business.arr_time.apply(categorizar_hora)
df_economy.drop(columns=['arr_time'], inplace=True)
df_business.drop(columns=['arr_time'], inplace=True)

In [ ]:
display(df_economy.head(10))
display(df_business.head(10))

## Pregunta 3

Genera un análisis exploratorio de los dataset entregados, un análisis univariado y bivariado. Prioriza los gráficos más importantes y entrega una conclusión a partir de
estos.

In [ ]:
df_business['business'] = 1 
df_economy['business'] = 0
df = pd.concat([df_business, df_economy], ignore_index=True)

In [ ]:
display(df.business.plot(kind='hist', title='Total business flights'))

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(
    go.Histogram(x=df_economy.date, name="Economy day of flight"),
    row=1, col=1
)
fig.append_trace(
    go.Histogram(x=df_business.date, name = 'Business day of flight'),
    row=2, col=1
)
fig.append_trace(
    go.Histogram(x=df.date, name = 'Total day of flight'),
    row=3, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(
    go.Histogram(x=df_economy.ch_code, name="Economy airline code"),
    row=1, col=1
)
fig.append_trace(
    go.Histogram(x=df_business.ch_code, name = 'Business airline code'),
    row=2, col=1
)
fig.append_trace(
    go.Histogram(x=df.ch_code, name = 'Total airline code'),
    row=3, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(
    go.Histogram(x=df_economy.arr_timeday, name="Economy arrive time"),
    row=1, col=1
)
fig.append_trace(
    go.Histogram(x=df_business.arr_timeday, name = 'Business arrive time'),
    row=2, col=1
)
fig.append_trace(
    go.Histogram(x=df.arr_timeday, name = 'Total arrive time'),
    row=3, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(
    go.Histogram(x=df_economy.dep_timeday, name="Economy departure time"),
    row=1, col=1
)
fig.append_trace(
    go.Histogram(x=df_business.dep_timeday, name = 'Business departure time'),
    row=2, col=1
)
fig.append_trace(
    go.Histogram(x=df.dep_timeday, name = 'Total departure time'),
    row=3, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(
    go.Histogram(x=df_economy['from'], name="Economy flight origin"),
    row=1, col=1
)
fig.append_trace(
    go.Histogram(x=df_business['from'], name = 'Business flight origin'),
    row=2, col=1
)
fig.append_trace(
    go.Histogram(x=df['from'], name = 'Total flight origin'),
    row=3, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(
    go.Histogram(x=df_economy.to, name="Economy flight destiny"),
    row=1, col=1
)
fig.append_trace(
    go.Histogram(x=df_business.to, name = 'Business flight destiny'),
    row=2, col=1
)
fig.append_trace(
    go.Histogram(x=df.to, name = 'Total flight destiny'),
    row=3, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(
    go.Histogram(x=df_business.time_taken, name="Business time taken in flight"),
    row=1, col=1
)
fig.append_trace(
    go.Histogram(x=df_economy.time_taken, name = 'Economy time taken in flight'),
    row=2, col=1
)
fig.append_trace(
    go.Histogram(x=df.time_taken, name = 'Total time taken in flight'),
    row=3, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.append_trace(
    go.Box(y=df_business.time_taken, name="Business time taken in flight"),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df_economy.time_taken, name="Economy time taken in flight"),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df.time_taken, name = 'Total time taken in flight'),
    row=1, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.append_trace(
    go.Box(y=df_business.price, name="Business flight price"),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df_economy.price, name="Economy flight price"),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df.price, name = 'Flight price'),
    row=1, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [ ]:
df.price.apply(lambda x: x if not f'{x}'.isnumeric() else None).dropna().index

In [ ]:
df.iloc[df.price.apply(lambda x: x if not f'{x}'.isnumeric() else None).dropna().index]

Se observan valores con problemas en la columna de precios, se presume que las comas estan mal puestas por lo que se eliminaran solamente las comas

In [ ]:
df.price = df.price.apply(lambda x: int(f'{x}'.replace(',', '')))

Se observan outliers en el tiempo de vuelo de los viajes, estos estan presentes tanto en los buelos economicos y de negocios, eliminaremos los outliers basados en el IQR

In [ ]:
def drop_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   not_outliers = df[~((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
   return not_outliers
print(df.shape)
df['time_taken'] = drop_outliers_IQR(df.time_taken)
df = df.dropna()
print(df.shape)
fig = make_subplots(rows=1, cols=1)
fig.append_trace(
    go.Box(y=df.time_taken, name = 'Total time taken in flight'),
    row=1, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

Podemos ver que se dismimuyen en gran cantidad los outliers de la muestra respecto al tiempo de vuelo y que los datos disminuyen aproximadamente en 2000 datos

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.append_trace(
    go.Box(y=df.price, name = 'Total price flight'),
    row=1, col=1
)

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

Tambien se observa que mejoro mucho la distribucion de los datos del precio de los datos

## Pregunta 4

Realiza un análisis de correlaciones entre las diferentes variables existentes, identificando cuáles son las variables más importantes para la predicción de la variable objetivo. Utiliza las técnicas vistas en el curso.

In [ ]:
sns.heatmap(df[['time_taken', 'stop', 'business', 'price']].corr(), annot=True, cmap='Reds')

Con lo anterior vemos que los vuelos de negocios tienen una marca muy importante sobre el precio del boleto, ademas de esto poco vemos sobre el tiempo de vuelo por si y con el si tuvieron paradas o no con el precio
Como no detectamos ninguna correlacion entre las variables que usaremos para predecir y muchas otras ya fueron eliminadas, proseguimos con el trabajo a realizar

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.append_trace(
    go.Box(y=df[df.dep_timeday == 'madrugada'].price, name = 'Precio con salida en la madrugada'),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df[df.dep_timeday == 'manana'].price, name = 'Precio con salida en la mañana'),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df[df.dep_timeday == 'tarde'].price, name = 'Precio con salida en la tarde'),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df[df.dep_timeday == 'noche'].price, name = 'Precio con salida en la noche'),
    row=1, col=1
)
fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

Con esto podemos observar el efecto del horario de salida en los precios, siendo las salidas que son en madrugada mucho mas baratos que el resto en general

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.append_trace(
    go.Box(y=df[df.arr_timeday == 'madrugada'].price, name = 'Precio con llegada en la madrugada'),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df[df.arr_timeday == 'manana'].price, name = 'Precio con llegada en la mañana'),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df[df.arr_timeday == 'tarde'].price, name = 'Precio con llegada en la tarde'),
    row=1, col=1
)
fig.append_trace(
    go.Box(y=df[df.arr_timeday == 'noche'].price, name = 'Precio con llegada en la noche'),
    row=1, col=1
)
fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

Analizando el precio en los horarios de llegada no obtenemos mucha mas informacion que la obtenida anteriormente

In [ ]:
fig = make_subplots(rows=1, cols=1)

for day in df.date.unique():
    fig.append_trace(
        go.Box(y=df[df.date == day].price, name = f'Precio en dia {day}'),
        row=1, col=1
    )

fig.update_layout(
    autosize=True,
    width=1000, height=800,
    margin=dict(
        l=50,
        r=100,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

Si bien se esperaba encontrar una diferencia notable en el precio del vuelo segun el dia, podemos dar cuenta que no es asi

## Pregunta 5

Realiza las transformaciones necesarias para realizar el modelamiento posterior y crea las variables que estimes convenientes con ayuda del análisis previo y la
expertise del negocio.
- La mayoria de las transformaciones necesarias ya han sido llevadas a cabo
- Se convertiran en one hot las columnas que requieran serlo

In [ ]:
df_dummy = df.copy()
for col in ['date', 'ch_code', 'from', 'to', 'dep_timeday', 'arr_timeday']:
    one_hot = pd.get_dummies(df_dummy[col], prefix=col.lower())
    df_dummy = df_dummy.drop(columns=col)
    df_dummy= df_dummy.join(one_hot)
df_dummy.head()

## Pregunta 6

Genera una función que encapsule el tratamiento de datos necesario, para entregar un dataset limpio y procesado a partir del dataset original.

In [ ]:
def normalize_data(df):
    def categorizar_hora(hora):
        if hora >= pd.to_datetime('00:00:00').time() and hora < pd.to_datetime('06:00:00').time():
            return 'madrugada'
        elif hora >= pd.to_datetime('06:00:00').time() and hora < pd.to_datetime('12:00:00').time():
            return 'manana'
        elif hora >= pd.to_datetime('12:00:00').time() and hora < pd.to_datetime('18:00:00').time():
            return 'tarde'
        else:
            return 'noche'
        
    def drop_outliers_IQR(df):
       q1=df.quantile(0.25)
       q3=df.quantile(0.75)
       IQR=q3-q1
       not_outliers = df[~((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
       return not_outliers

    df.drop(columns=['num_code', 'airline'], inplace=True)
    df.stop = df.stop.apply(lambda x: 0 if x == 'non-stop' else int(x[0]))
    df.date = df.date.apply(lambda x: x.date().strftime("%A"))
    df['hours'] = df.time_taken.apply(lambda x: x[:x.index('h')])
    df['minutes'] = df.time_taken.apply(lambda x: x[x.index('h')+1:x.index('m')])
    df.time_taken = df.hours.apply(lambda x: int(float(x)*60) if x.strip() != '' else 0) + \
        df.minutes.apply(lambda x: int(x) if x.strip() != '' else 0)
    df['dep_timeday'] = df.dep_time.apply(categorizar_hora)
    df['arr_timeday'] = df.arr_time.apply(categorizar_hora)
    df.drop(columns=['hours', 'minutes', 'dep_time', 'arr_time'], inplace=True)
    
    df.price = df.price.apply(lambda x: int(f'{x}'.replace(',', '')))
    df['time_taken'] = drop_outliers_IQR(df.time_taken)
    df_dummy = df.dropna().copy()
    for col in ['date', 'ch_code', 'from', 'to', 'dep_timeday', 'arr_timeday']:
        one_hot = pd.get_dummies(df_dummy[col], prefix=col.lower())
        df_dummy = df_dummy.drop(columns=col)
        df_dummy= df_dummy.join(one_hot)
    return df_dummy
df = normalize_data(df)

In [ ]:
df_dummy = df

## Pregunta 7

Elige al menos 3 modelos candidatos para resolver el problema. A partir de esto, genera un conjunto de entrenamiento y prueba, para luego entrenar los diferentes
modelos.

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
x = df_dummy[list(set(df_dummy.columns)-set(['price']))]
y = df_dummy['price']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21)

## Pregunta 8

Elige una grilla de hiperparametros y luego optimízalos, buscando la mejor combinación para cada grilla. Guardar los modelos entrenados.

Para esto usaremos regresion lineal ridge , un arbol de regresion y random forest para regresion

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5, 10],
    'solver': ['auto', 'svd', 'cholesky', 'sparse_cg', 'sag', 'saga'],
}
ridge_model = Ridge()

# Realizar la búsqueda de hiperparámetros utilizando validación cruzada
grid_search_ridge = GridSearchCV(ridge_model, param_grid, cv=10, verbose=1)
grid_search_ridge.fit(X_train, y_train)
# Obtener los resultados de la búsqueda
best_params_ridge = grid_search_ridge.best_params_
best_score_ridge = grid_search_ridge.best_score_
print("Mejores hiperparámetros:", best_params_ridge)
print("Precisión con los mejores hiperparámetros:", best_score_ridge)
grid_search_ridge.best_estimator_

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
param_grid = {
    'random_state': [0],
    'max_depth': [5, 10, 30, 100, 200],
    'criterion': ["squared_error", "absolute_error", "poisson"],
    'min_samples_split': [15, 20]
}
dtr_model = DecisionTreeRegressor(random_state=0)

# Realizar la búsqueda de hiperparámetros utilizando validación cruzada
grid_search_dtr = GridSearchCV(dtr_model, param_grid, cv=10, verbose=1)
grid_search_dtr.fit(X_train, y_train)
# Obtener los resultados de la búsqueda
best_params_dtr = grid_search_dtr.best_params_
best_score_dtr = grid_search_dtr.best_score_
print("Mejores hiperparámetros:", best_params_dtr)
print("Precisión con los mejores hiperparámetros:", best_score_dtr)
grid_search_dtr.best_estimator_

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 300 ],
    'criterion': ["squared_error", "absolute_error", "poisson"],
    'max_depth': [5, 10, 30, 100, 200],
    'min_samples_split': [15, 20]
}
rf_model = RandomForestRegressor(random_state=0)

# Realizar la búsqueda de hiperparámetros utilizando validación cruzada
grid_search_rf = GridSearchCV(rf_model, param_grid, cv=10, verbose=1)
grid_search_rf.fit(X_train, y_train)
# Obtener los resultados de la búsqueda
best_params_rf = grid_search_rf.best_params_
best_score_rf = grid_search_rf.best_score_
print("Mejores hiperparámetros:", best_params_rf)
print("Precisión con los mejores hiperparámetros:", best_score_rf)
grid_search_rf.best_estimator_

## Pregunta 9

Define al menos 3 métricas para evaluar los modelos entrenados y genera gráficos de comparación. Elige un baseline para ver qué tan buena es tu opción respecto a
ese baseline y concluye.

In [ ]:
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

## Pregunta 10

Genera una conclusión final respecto a qué tan útiles son los resultados encontrados para resolver el problema propuesto y define cuáles podrían ser los próximos pasos
para el proyecto.